# A/B testing dataset

## Exploratory dataset analysis & cleaning the data - for future analysis 

---

In [19]:
import pandas as pd

df = pd.read_csv("ab_data.csv")
df2 = pd.read_csv("countries.csv")

In [20]:
print(df['user_id'].count())

294478


In [21]:
print(df2['user_id'].count())

290584


In [22]:
df['user_id'].nunique()

290584

In [23]:
df2['user_id'].nunique()

290584

#### **Second dataframe which contains `user_id` and `countries` has less users than first dataframe does, which tells us that the first dataframe contains duplicates that are worth getting rid of .**

In [24]:

df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('s')
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48,control,old_page,0
1,804228,2017-01-12 08:01:45,control,old_page,0
2,661590,2017-01-11 16:55:06,treatment,new_page,0
3,853541,2017-01-08 18:28:03,treatment,new_page,0
4,864975,2017-01-21 01:52:26,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38,control,old_page,0
294474,945152,2017-01-12 00:51:57,control,old_page,0
294475,734608,2017-01-22 11:45:03,control,old_page,0
294476,697314,2017-01-15 01:20:28,control,old_page,0


#### **Converting `timestamp` column into date format , since it wasn't , also round it to the seconds , to make the column look better**

In [25]:
df.isna().sum()
df2.isna().sum()

df.dtypes

user_id                  int64
timestamp       datetime64[ns]
group                   object
landing_page            object
converted                int64
dtype: object

In [26]:
ab_glued_data = df.merge(df2, how ="inner" , on='user_id')
ab_glued_data.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
country         0
dtype: int64

#### **Merging two dataframes to get a new column in the main data frame - `"country"`**

In [27]:
ab_glued_data = ab_glued_data.drop_duplicates(subset=['user_id'])

In [28]:
ab_glued_data.shape

(290584, 6)

In [29]:
ab_glued_data

,user_id,timestamp,group,landing_page,converted,country
0,851104,2017-01-21 22:11:48,control,old_page,0,US
1,804228,2017-01-12 08:01:45,control,old_page,0,US
2,661590,2017-01-11 16:55:06,treatment,new_page,0,US
3,853541,2017-01-08 18:28:03,treatment,new_page,0,US
4,864975,2017-01-21 01:52:26,control,old_page,1,US
...,...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38,control,old_page,0,US
294474,945152,2017-01-12 00:51:57,control,old_page,0,US
294475,734608,2017-01-22 11:45:03,control,old_page,0,US
294476,697314,2017-01-15 01:20:28,control,old_page,0,US


In [30]:
ab_glued_data.groupby(['group'])['user_id'].count()

group
control      145232
treatment    145352
Name: user_id, dtype: int64

#### **Removing duplicates already in the merged table to save time, and do not delete them in each one. Now the groups look even, and conversion will be more accurate**

In [31]:
pd.crosstab(ab_glued_data['group'], ab_glued_data['landing_page'])

landing_page,new_page,old_page
group,,
control,1006,144226
treatment,144314,1038


In [32]:
conversion = pd.Series(ab_glued_data.groupby(['group'])['converted'].mean() * 100)

print(conversion.round(2).apply(lambda x: str(x) + " %"))

group
control      12.03 %
treatment    11.88 %
Name: converted, dtype: object


#### **Now we are comparing conversion between 2 even groups, but difference not that huge , it may be statistical , but not practical - `0.15%` it may not be worth so much effort.**

---

In [35]:
ab_glued_data.to_csv(r"C:\Users\lelik\Desktop\vs\AlexIrnazarovPet_projects\AB-testing\final_data\final_data.csv", index=False)

#### **Saving final `data.csv` for future SQL analysis**

## Business Recommendations

### Recommendation
**Keep the old page design.** Despite development efforts on the new page, data shows the original design maintains slightly better performance.

### Rationale
1. **Performance comparison**: Old page shows 12.03% conversion vs new page's 11.88%
2. **Sample size reliability**: With ~145,000 users per group, the measurement is stable
3. **Practical consideration**: 0.15% difference is minimal, but even this small advantage favors the existing solution
4. **Cost-benefit analysis**: Investment in new page development did not yield improvement - no justification for implementation costs

### Next Steps
1. **Immediate action**: Discontinue rollout of new page design
2. **Learning opportunity**: Analyze what elements of the old page drive better conversion
3. **Future optimization**: Use insights to design more targeted experiments focused on specific high-impact elements